Wireshark Project: Analyzing a Basic HTTPS Request**
Objective
Capture and analyze the key elements of an HTTPS connection initiated by visiting a secure website in a browser, focusing on the TLS handshake and initial request details.

Step-by-Step Plan
1. Preparation
Interface: Identify the active network interface in Wireshark (likely Wi-Fi).

Display Filter (optional):
Apply a filter before or after capture:


tls
or


tcp.port == 443

2. Start Packet Capture
Open Wireshark.

Select your network interface.

Begin capturing traffic.

3. Trigger the HTTPS Request
Open a web browser.

Visit a well-known secure website 

Wait 5–10 seconds for the page to load fully.

4. Stop the Capture
Return to Wireshark and stop the packet capture after loading completes.

Analysis Plan
5. Locate the HTTPS Traffic
Use the filter:


tls.handshake

Look for:

Client Hello

Server Hello

Certificate

Encrypted Handshake Message

6. Capture Key Details

For documentation in the notebook, record:

Source and destination IP addresses.

SNI (Server Name Indication) – domain the client is connecting to.

TLS version (e.g., TLS 1.3).

Cipher suite negotiated.

Certificate issuer and validity period.

7. Quick Observations (Markdown Section)

TLS is encrypted—you will not see the actual HTTP request content (unlike HTTP).

The handshake shows who the client is connecting to and what encryption was agreed on.

Certificate inspection reveals trust relationships (e.g., signed by DigiCert or Let's Encrypt).



## 🔍 Wireshark TLS Analysis: Client Hello to CNN.com

### 📌 Frame 222 Summary

**Action**: Captured initial `Client Hello` during HTTPS request to `https://cnn.com`

- **Frame Number**: 222  
- **Captured Bytes**: 515 bytes  
- **Interface**: `\Device\NPF_{EAF66F3F-...}`  
- **Network Layer**:  
  - **Source IP**: `2600:1700:a3a0:d90:813:a83e:73bf:7fba`  
  - **Destination IP**: `2a04:4e42:200::773` (`cnn.com`)  
  - **Protocol**: IPv6  
- **Transport Layer**:  
  - **Protocol**: TCP  
  - **Source Port**: 10922  
  - **Destination Port**: 443 (`https`)  
  - **TCP Segments**: Reassembled from #221 and #222  
- **TLS Layer**:  
  - **Version**: TLS 1.3  
  - **Record Type**: Handshake  
  - **Handshake Type**: Client Hello  
  - **Length**: 1776 bytes  
  - **SNI (Server Name Indication)**: `cnn.com`  

### 🔎 Notes

This packet represents the **start of the TLS handshake**. The client (your system) initiated a secure connection to `cnn.com`. While the record layer reports **TLS 1.0** (0x0301) for backward compatibility, the actual handshake content indicates **TLS 1.3**—a common behavior in TLS negotiation.

The handshake is encrypted beyond this point, meaning no content such as HTTP requests or page data is visible without decryption keys (by design in HTTPS).

---




## 🔁 TLS Server Hello and Encrypted Response (Frame 255)

### 📌 Frame 255 Summary

**Action**: Captured `Server Hello` in response to the earlier `Client Hello` (Frame 222)

- **Frame Number**: 255  
- **Captured Bytes**: 1414 bytes  
- **Interface**: `\Device\NPF_{EAF66F3F-...}`  
- **Network Layer**:  
  - **Source IP**: `2a04:4e42:200::773` (`cnn.com`)  
  - **Destination IP**: `2600:1700:a3a0:d90:...` (Your machine)  
  - **Protocol**: IPv6  
- **Transport Layer**:  
  - **Protocol**: TCP  
  - **Source Port**: 443 (`https`)  
  - **Destination Port**: 10922  
- **TLS Layer**:
  - **Record 1**:  
    - Type: Handshake  
    - Subtype: **Server Hello**  
    - TLS Version: 1.3 (recorded as 1.2 for backward compatibility: `0x0303`)  
    - Length: 122 bytes  
  - **Record 2**:  
    - Type: Change Cipher Spec  
    - Indicates encryption is about to begin  
  - **Record 3**:  
    - Type: Encrypted Application Data  
    - Length: 36 bytes  
    - Meaning: This data is encrypted and cannot be interpreted without session keys  
    - Protocol Detected: HTTPS (via ALPN in earlier packets)

---

### 🔎 Analyst Notes

- The **TLS version still appears as 1.2 (`0x0303`)**, but this is a normal quirk of TLS 1.3 for backward compatibility.
- The **Server Hello** completes the negotiation of cipher suites and session keys.
- The **Change Cipher Spec** marks the shift to encrypted communication.
- The final **Encrypted Application Data** confirms the session has begun secure transfer.

From this point forward, all data—including the actual HTTP `GET` request for the CNN webpage—is fully encrypted.



TLS Handshake Analysis (Frame 82)
➡️ Server IP & Domain:
52.182.143.211 — v10.events.data.microsoft.com

➡️ TLS Version:
TLS 1.2 (noted as 0x0303)

➡️ Cipher Suite Negotiated:
TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384

ECDHE: Ephemeral Elliptic Curve Diffie-Hellman for forward secrecy

RSA: Server authentication

AES_256_GCM: Symmetric encryption with authentication

SHA384: Hashing algorithm used in integrity checks

➡️ Certificate Issuer & Chain:

The server certificate is signed by the Microsoft Secure Server CA 2011

That certificate is part of a chain validated by Microsoft Root Certificate Authority 2011

Both certificates use RSA with SHA-256 for signing

➡️ Certificate Validity & Trust Indicators:

Validity dates not shown in your copy-paste, but present in Wireshark’s X.509 Certificate field

The issuer's Common Name (CN) clearly identifies the Microsoft certificate authority — a trusted CA

Extensions such as Basic Constraints, Authority Info Access, and Key Usage are present and typical

➡️ Notes:

This handshake also included Application-Layer Protocol Negotiation (ALPN) with value h2 (HTTP/2), showing the server prefers HTTP/2

Additional extensions like extended_master_secret and renegotiation_info help prevent known TLS attacks

The Server Hello Done message finalizes the server’s part of the handshake



**Conclusion**

This project provided a focused investigation into a standard HTTPS connection using Wireshark, capturing and analyzing the TLS handshake between a client system and a web server (v10.events.data.microsoft.com). The analysis confirmed the negotiation of TLS 1.3, the use of a strong cipher suite (TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384), and a valid certificate chain anchored by Microsoft’s Root CA.

By inspecting individual packets, we confirmed:

The use of Server Name Indication (SNI) in the Client Hello

The negotiation of cryptographic parameters and TLS extensions

The presentation of X.509 certificates and associated metadata

The completion of the server handshake with Server Hello Done


